In [154]:
import warnings
import pandas as pd
from Plots import plot_political_label_counts, custom_show_fig, plot_box_plot_rating_politics, plot_political_outliers, plot_box_plot_confidence_politics
from FilmCollector import Film_collector
import asyncio
from Utils import extract_imdb_id, json_to_dataframe, predict_reviews, split_by_politics, filter_outside_range, print_prediction_results, sort_by_confidence, df_rating_is
import nest_asyncio
import json
from string import Template
from IPython.display import Image, display, HTML
warnings.filterwarnings('ignore')
import os

### Set a film

Put the link to the IMDb page in order to fetch analyse the film, 
FORCE_UPDATE will re fetch the data is set true, even if the data is stored.

In [155]:
FILM_LINK = 'https://m.imdb.com/title/tt7713068/?ref_=tt_sims_tt_t_2'
FORCE_UPDATE = False

In [156]:
nest_asyncio.apply()
collector = Film_collector()
async def get_film_data():
    return await collector.get_all(extract_imdb_id(FILM_LINK), FORCE_UPDATE)

film_data_json, df_reviews = asyncio.get_event_loop().run_until_complete(get_film_data())

Fetching film data from IMDB
Fetching reviews from IMDB
Fetching reviews for film tt7713068


In [157]:
df_reviews.head()

,review_title,review_body,rating,review_date
0,"Entertaining at times, but is all over the place",Birds of Prey: and the Fantabulous Emancipatio...,5.0,6 February 2020
1,Entertaining but a bit messy.,Birds of prey shows that DC has finally righte...,7.0,8 February 2020
2,"This is not a ""man hating"" movie...",Is this a good movie? Not really. Is it a bad ...,7.0,10 March 2020
3,Robbie and McGregor are brilliant even if the ...,,7.0,5 February 2020
4,MARGOT STEALS THE SHOW.,"Funny, non stop action, kinda all over the pla...",7.0,12 February 2020


In [158]:
df_film_data = json_to_dataframe( json.loads(film_data_json))
df_film_data.head()
image_url = df_film_data['Poster'].iloc[0]
display(df_film_data)
display(Image(url=image_url, width=300, height=400))

,Type,Name,URL,Poster,Description,ContentRating,DatePublished,Keywords,Duration,RatingCount,RatingValue,Genres,Actors,Directors,Creators
0,Movie,Birds of Prey and the Fantabulous Emancipation...,https://www.imdb.com/tt7713068/,https://m.media-amazon.com/images/M/MV5BMzQ3NT...,"After splitting with the Joker, Harley Quinn j...",14,2020-02-07,"harley quinn character,female protagonist,swal...",PT1H49M,266031,6.1,"Action, Comedy, Crime","Margot Robbie, Rosie Perez, Mary Elizabeth Win...",Cathy Yan,"Christina Hodson, Paul Dini, Bruce Timm"


# Review Prediction

In [159]:
df_reviews_predicted = predict_reviews(df_reviews)
print(len(df_reviews_predicted))
df_reviews_predicted.head(10)

2715


,review_title,document,politics,confidence,rating,text_length
0,"Entertaining at times, but is all over the place","Entertaining at times, but is all over the pla...",neutral,0.600090,5.0,2078
1,Entertaining but a bit messy.,Entertaining but a bit messy. Birds of prey sh...,neutral,0.518379,7.0,1739
2,"This is not a ""man hating"" movie...","This is not a ""man hating"" movie... Is this a ...",left,0.417607,7.0,677
3,Robbie and McGregor are brilliant even if the ...,Robbie and McGregor are brilliant even if the ...,neutral,0.852925,7.0,74
4,MARGOT STEALS THE SHOW.,"MARGOT STEALS THE SHOW. Funny, non stop action...",neutral,0.914252,7.0,115
5,Fun movie!,Fun movie! Margot Robbie is the perfect Harley...,left,0.526293,7.0,365
6,Boring and forgettable,Boring and forgettable While Margot Robbie is ...,right,0.974649,4.0,413
7,I don't understand the hate at all,I don't understand the hate at all I would lik...,neutral,0.975640,8.0,2503
8,"A -Completely- Fun, Big "" Antiheroine "" Romp F...","A -Completely- Fun, Big "" Antiheroine "" Romp F...",right,0.631699,7.0,5133
9,It made me sleepy.,"It made me sleepy. I'm glad I had my phone, th...",neutral,0.979808,3.0,865


## Neutral

In [160]:
df_reviews_neutral = split_by_politics(df_reviews_predicted, 'neutral')
print(len(df_reviews_neutral))
df_reviews_neutral.head(10)

1811


,review_title,document,politics,confidence,rating,text_length
0,"Entertaining at times, but is all over the place","Entertaining at times, but is all over the pla...",neutral,0.600090,5.0,2078
1,Entertaining but a bit messy.,Entertaining but a bit messy. Birds of prey sh...,neutral,0.518379,7.0,1739
3,Robbie and McGregor are brilliant even if the ...,Robbie and McGregor are brilliant even if the ...,neutral,0.852925,7.0,74
4,MARGOT STEALS THE SHOW.,"MARGOT STEALS THE SHOW. Funny, non stop action...",neutral,0.914252,7.0,115
7,I don't understand the hate at all,I don't understand the hate at all I would lik...,neutral,0.975640,8.0,2503
9,It made me sleepy.,"It made me sleepy. I'm glad I had my phone, th...",neutral,0.979808,3.0,865
11,"A great all-girls group film, Birds of Prey ea...","A great all-girls group film, Birds of Prey ea...",neutral,0.507674,6.0,5009
12,What is people's problem with this one?,What is people's problem with this one? Overal...,neutral,0.931872,7.0,459
13,Better than SUICIDE SQUAD,Better than SUICIDE SQUAD Did you catch the 20...,neutral,0.505833,7.0,3779
15,Bird-Brained,Bird-Brained GRADE: C-\n\nTHIS FILM IS NOT REC...,neutral,0.747230,4.0,3929


Neutral review outliers, higher change of classification error with rating 1, and 10

In [161]:
df_purge_neutral_filtered = filter_outside_range(df_reviews_neutral, min_rating=1, max_rating=10)
print(len(df_purge_neutral_filtered))
print_prediction_results(sort_by_confidence(df_purge_neutral_filtered))

532
Document title: The script is messy
Document: The script is messy 
Confidence: 0.999954342842102
Document title: I'm sorry, but the movie is disgusting
Document: I'm sorry, but the movie is disgusting I was hoping to correct the main movie mistakes but unfortunately I was disappointed
Confidence: 0.9996870756149292
Document title: So much fun and perfect soundtrack
Document: So much fun and perfect soundtrack Ok the plot it's not the best, but none of the hero movies have a decent plot. I guess it all boils down to ur experience and I sure and plenty of fun watching it.

The soundtrack slaps so hard and the movie is soooo colorful. Loved it
Confidence: 0.9994699358940125
Document title: It is a fun action flick calm down
Document: It is a fun action flick calm down This movie is a great time and Margo Robbie is a great lead, the fight choreography is great and there isn't really a boring part. I'm honestly giving it ten stars cause people need to stop hating so much.
Confidence: 0.

## Left

In [162]:
df_reviews_left = split_by_politics(df_reviews_predicted, 'left')
print(len(df_reviews_left))
df_reviews_left.head(10)

540


,review_title,document,politics,confidence,rating,text_length
2,"This is not a ""man hating"" movie...","This is not a ""man hating"" movie... Is this a ...",left,0.417607,7.0,677
5,Fun movie!,Fun movie! Margot Robbie is the perfect Harley...,left,0.526293,7.0,365
17,Silly and Entertaining,"Silly and Entertaining In Gotham City, the Jok...",left,0.359249,6.0,1077
24,underated,underated I dont understand why so hate? i lik...,left,0.840520,7.0,83
33,"Fun movie, like Aquaman.","Fun movie, like Aquaman. It's a comic book mov...",left,0.702382,7.0,401
37,Movie completely disrespects the source,Movie completely disrespects the source It's t...,left,0.447482,3.0,267
39,Huge fan actually watched the movie,Huge fan actually watched the movie All the ac...,left,0.615519,7.0,117
42,Forgettable Harley Quinn Movie,Forgettable Harley Quinn Movie The movie is no...,left,0.833975,6.0,933
43,Horrendous and forced,Horrendous and forced Only Margot Robbie saves...,left,0.659096,1.0,133
44,"A fast-paced, violent, and sometimes disjointe...","A fast-paced, violent, and sometimes disjointe...",left,0.399706,7.0,5005


In [163]:
df_reviews_left_negative = df_rating_is(df_reviews_left, 1)
print(len(df_reviews_left_negative))
print_prediction_results(sort_by_confidence(df_reviews_left_negative))

136
Document title: Silly beyond sense and not worth a cinema ticket
Document: Silly beyond sense and not worth a cinema ticket This is one film to not spend hard earned cash on. Wait for it to appear on tv instead. Even then its debatable whether to watch or not.

Whilst with Harley its faithful to the batman the animated series, the supporting characters it's a travesty. They've taken the names and butchered the identities ao they dont resemble any of the comic characters at all.
Confidence: 0.9969140291213989
Document title: Lifeless because...
Document: Lifeless because... 
Confidence: 0.9960695505142212
Document title: Zzz...
Document: Zzz... I'm writing this instead of watching the movie. 'Nuff said?

'Suicide Squad' was entertaining. This? Not at all.

No idea how this got a 60.

PS - It actually got WORSE after I left my review. I will never get that portion of my life back. I award them zero points; and may God have mercy on their souls.
Confidence: 0.992870032787323
Document 

In [164]:
df_reviews_left_positive = df_rating_is(df_reviews_left, 10)
print(len(df_reviews_left_positive))
print_prediction_results(sort_by_confidence(df_reviews_left_positive))

41
Document title: Marvel Stans hating on everything
Document: Marvel Stans hating on everything It was one of the best movie in 2020 so far. Grab a popcorn and watch it!!!🍿
Confidence: 0.982801079750061
Document title: One of the best for DC
Document: One of the best for DC 
Confidence: 0.9815887212753296
Document title: One of the best movies I've seen in some time
Document: One of the best movies I've seen in some time 
Confidence: 0.9750516414642334
Document title: I saw the Movie !
Document: I saw the Movie ! Hello friends let me say something, ok ? Yes it is a messy movie but it is because its Harley Queen's story she is the narrator ! And that makes a messy movie a fine one from my opinion. From the last movies that Dc had put up I say from WW and Justice League Green Lantern Joker 2019 and this one They are coming back 4 real. 10/10 for my childhood and for DC keep it up this way !
Confidence: 0.9610421061515808
Document title: Sisters getting. It. Done.
Document: Sisters getti

## Right

In [165]:
df_reviews_right = split_by_politics(df_reviews_predicted, 'right')
print(len(df_reviews_right))
df_reviews_right.head(10)

364


,review_title,document,politics,confidence,rating,text_length
6,Boring and forgettable,Boring and forgettable While Margot Robbie is ...,right,0.974649,4.0,413
8,"A -Completely- Fun, Big "" Antiheroine "" Romp F...","A -Completely- Fun, Big "" Antiheroine "" Romp F...",right,0.631699,7.0,5133
10,I liked it,I liked it I liked it. It was entertaining and...,right,0.743502,8.0,370
14,"Funny, sexy, and psycho.","Funny, sexy, and psycho. Have you ever watched...",right,0.869338,7.0,398
16,Really enjoyed this,Really enjoyed this I've been a fan of Harley ...,right,0.506520,8.0,507
20,"Finally, a superhero movie I (almost) like","Finally, a superhero movie I (almost) like The...",right,0.812875,NaN,1416
41,Maybe watch it first before reviewing it,Maybe watch it first before reviewing it It's ...,right,0.492039,10.0,942
49,Honest display of an era of confused feminism,Honest display of an era of confused feminism ...,right,0.612156,7.0,568
56,Better than 'Suicide Squad'... Barely,Better than 'Suicide Squad'... Barely Margot R...,right,0.597778,7.0,1129
57,It sucks,It sucks Not impressed with this film. The nov...,right,0.563526,3.0,217


In [166]:
df_purge_right_negative = df_rating_is(df_reviews_right, 1)
print(len(df_purge_right_negative))
print_prediction_results(sort_by_confidence(df_purge_right_negative))

60
Document title: Absolutely annoying to watch
Document: Absolutely annoying to watch This movie breaks the one rule of cinema: show, don't tell. It's nothing but narration. The pacing is all over the place, and they butcher Harley's character. It was everything I feared it would be.
Confidence: 0.9988453388214111
Document title: A hot mess, and not in the way you want it to be
Document: A hot mess, and not in the way you want it to be Where do I begin? This movie was a WOMBAT - a Waste Of Money, Brains and Talent (or Time, depending on your perspective). The director was clearly over her head and needs more experience before helming a big-budget action film.

The first error was not developing the supporting characters. Time was spent with the backstories for Huntress and Montoya, but virtually nothing for Black Canary. No explanation of her powers (how she got them, why she passes out when she uses them, etc.) That could have been handled in a 60-to-90 second montage.

Second error?

In [167]:
df_purge_right_positive = df_rating_is(df_reviews_right, 10)
print(len(df_purge_right_positive))
print_prediction_results(sort_by_confidence(df_purge_right_positive))

39
Document title: Harley Quinn is back
Document: Harley Quinn is back Birds of prey was 10 times better the Sucide squad much more funny and the action scene were better put together.
Confidence: 0.9982439279556274
Document title: NOOOOO my egg sandwich *Spoiler Alert*
Document: NOOOOO my egg sandwich *Spoiler Alert* 
Confidence: 0.9948039650917053
Document title: best of febuary hands down
Document: best of febuary hands down Okay so let me start by saying i knew that i was gonna like this movie however what i did not expect was to absolutely love it it was action packed to the more and funny with a dash of twisted one of the best ive seen so far this year you get to know every character personally and all the ladies are spectacular and all the villans are absuloutley wicked
Confidence: 0.9866146445274353
Document title: Pretty good
Document: Pretty good Would give it a 7 or maybe 6, but I'm just compensating for all the 1's with no reason at all.
Confidence: 0.946000337600708
Docume

# PLots

In [168]:
custom_show_fig(plot_political_label_counts(df_reviews_predicted).update_layout(width=800, height=600))

In [169]:
custom_show_fig(plot_political_outliers(df_reviews_predicted).update_layout(width=800, height=600))

In [170]:
custom_show_fig(plot_box_plot_rating_politics(df_reviews_predicted).update_layout(width=800, height=600))

In [171]:
custom_show_fig(plot_box_plot_confidence_politics(df_reviews_predicted).update_layout(width=800, height=600))

# New score calculation
### Partial score
Score using just the reviews score in order to calculate a new value.

In [172]:
def dataframe_to_json_context(df_predicated, df_film):
    df_left = df_predicated[df_predicated['politics'] == 'left']
    df_right = df_predicated[df_predicated['politics'] == 'right']
    df_neutral = df_predicated[df_predicated['politics'] == 'neutral']
    df_left_1 = df_left[df_left['rating'] == 1]
    df_left_1.loc[:, 'politics'] = 'left 1'
    df_left_10 = df_left[df_left['rating'] == 10]
    df_left_10.loc[:, 'politics'] = 'left 10'
    df_right_1 = df_right[df_right['rating'] == 1]
    df_right_1.loc[:, 'politics'] = 'right 1'
    df_right_10 = df_right[df_right['rating'] == 10]
    df_right_10.loc[:, 'politics'] = 'right 10'
    df_right_filtered = df_right[~df_right['rating'].isin([1, 10])]
    df_left_filtered = df_left[~df_left['rating'].isin([1, 10])]
    
    df_concatenated = pd.concat([
        df_neutral,
        df_right_filtered,
        df_left_filtered,
        df_left_1,
        df_left_10,
        df_right_1,
        df_right_10,
    ], ignore_index=True)
    
    df_concatenated = df_concatenated.dropna(subset=['rating'])
    data = df_concatenated[['rating', 'politics', 'confidence']].to_dict(orient='records')
    context = {
        'data': json.dumps(data),
        'imdb_score': str(df_film['RatingValue'][0]),
        'imdb_doc_count': str(df_film['RatingCount'][0]),
    }
    return context
    

  

json4 = dataframe_to_json_context(df_reviews_predicted, df_film_data)

file_path = os.getcwd() + '/document_creator/movie_score.html'
with open(file_path, 'r') as file:
    src = Template(file.read())
html = src.safe_substitute(json4)
display(HTML(html))